## Pythonic Codes

파이쏘닉 코드란, `파이썬 스러운` 코드를 의미한다. 프로그래밍에서 관용구는 특정 작업을 수행하기 위해 코드를 작성하는 특별한 방법이다.

이 관용구는 언어에 따라 다르기 마련인데, 파이썬의 고유한 메커니즘을 따른 코드를 **파이썬스럽다**고 한다.

### Pythonic 한 코드를 작성해야 하는 이유?

일반적으로 관용적인 방식으로 코드를 작성했을 때 성능이 좋도 코드도 짧으며 이해가 쉽다.

또한 동일한 패턴과 구조에 익숙해지면 실수를 줄이고 본질에 집중할 수 있다.

그 근본적인 아이디어들에 대해 살펴보자.

## Index와 슬라이스

### 파이썬의 인덱싱
파이썬의 일부 데이터나 구조에서 요소에 접근할 때 주로 인덱스를 사용하곤 한다.

파이썬은 다른 언어완 달리, <span style="background-color: #FFFB00;">음수를 사용하여 데이터의 끝에서 부터 접근</span>이 가능하다.

In [1]:
nums = (1, 1, 2, 3, 5, 8, 13, 21)
print(nums[-1])
print(nums[-5])

21
3


### 파이썬의 슬라이싱
또한 파이썬은 `slice`를 사용하여 **특정 구간의 요소**를 구할 수 있다.

슬라이싱에는 `[시작 인덱스, 끝 인덱스, 간격]`을 설정할 수 있다.

단, 슬라이스의 시작 인덱스는 포함되지만, 끝 인덱스는 제외되며, 시작 인덱스나 끝 인덱스를 작성하지 않을 경우 **처음부터** 혹은 **끝까지**로 인식하며, 동작한다.

In [2]:
print(nums[2:4])  # 2번째 인덱스부터 4번째 인덱스 직전까지
print(nums[:-3])  # 뒤에서부터 세 번째 인덱스 직전까지
print(nums[::2])  # 처음부터 끝까지 2개 인덱스씩 건너뛰면서

(2, 3)
(1, 1, 2, 3, 5)
(1, 2, 5, 13)


### 자체 시퀀스 생성
이런 파이썬의 인덱싱과 슬라이싱은 `__getitem___`이라는 매직 메소드로 동작한다.

> magic method, 혹은 special method라고 불리는 매직 메소드는 파이썬에서 내부적으로 구현된 메소드들을 일컫는다. <p>
https://rszalski.github.io/magicmethods/ 에 가이드가 있고, 앞뒤로 double under score로 감싸져 있는 것이 특징이다.

`__getitem__`은 `myobject[key]`와 같은 형태를 사용할 때 사용되는 메소드이다.

특히 `__getitem__`과 `__len__`을 사용하여 시퀀스나 이터러블 객체를 만들지 않고 키를 통해 객체의 특정 요소를 가져올 수 있다.

## 컨텍스트 관리자
컨텍스트 관리자(Context Manager)는 크게 두 가지 경우에 유용한 파이썬의 기능이다.

### 리소스 관리
일반적으로 파일이나 소켓 연결을 열었을 때, 할당된 리소스를 모두 해제해주어야 하는데, 

이 과정에서 생각하지 못한 예외나 오류가 발생할 수 있다.

이를 사전에 모두 처리하는 것은 어렵지만, 파이썬에서는 **with문**을 사용해 Pythonic하게 구현할 수 있다.

```py
with open(filename) as fd:
    process_file(fd)
```

블록의 마지막이 실행되고 나면 컨텍스트가 종료되며, 오류가 있더라도 종료되므로 안전하게 실행할 수 있다.

### 코드 분리
주요 동작의 전후에 작업을 실행하려고 할 때나 독립적으로 코드를 분리해야 할 때가 있다.

### 컨텍스트 관리자 구현
컨텍스트 관리자는 일반적으로 `__enter__`와 `__exit__`두 개의 매직 메소드만 구현하면 되지만, `contextlib`모듈을 사용하여 더 쉽게 구현할 수도 있다.

`__enter__`메소드가 호출되면 새로운 컨텍스트로 진입하게 되며, 컨텍스트의 마지막 문장이 끝나면 컨텍스트가 종료되며 처음 호출한 컨텍스트 관리자 객체의 `__exit__`메소드를 호출한다.

```py
class dbhandler_decorator(contextlib.ContextDecorator):
    def __enter__(self):
        start_database()

    def __exit__(self, ext_type, ex_value, ex_traceback):
        stop_database()

@dbhandler_decorator()
def offline_backup():
    run("pg_dump database")
```

> 이렇게 컨텍스트 관리자를 데코레이터로 지정해주게 되면 offline_backup 함수를 호출만 하더라도 컨텍스트 관리자 안에서 자동으로 실행됨

## 프로퍼티, 속성과 객체 메소드의 다른 타입들
프로퍼티에 대해 알아보기 전에 파이썬에서의 밑줄에 대해 잠깐 살펴보자.

### 파이썬에서의 밑줄
일반적으로 파이썬에서의 변수나 메소드 이름 앞의 밑줄은 `private`를 의미한다.
> 단, private를 의미만 할 뿐 private하게는 만들 수 없다.

그런데 밑 줄 두개는 전혀 다른 `name mangling`이라는 것을 실행한다.

`name mangling`이란 말 그대로 이름을 만드는 것인데, `_<class-name>__<attribute-name>`형태의 이름을 만든다.

이런식으로 네이밍하는 이유는 **클래스가 여러 번 확장되더라도 충돌 없이 오버라이드를 하기 위한 것**이다.

따라서 가끔 `name mangling`이 전혀 다른 이름을 만들어내어 본래의 이름으로는 해당 속성에 접근할 수 없게 되기 때문에

밑줄 두 개를 작성하는 것이 정말 `private`하게 만든다고 생각하는 사람이 있다. 이는 매우 잘못된 생각이라고 한다.

```py
class Connector:
    def __init__(self, source):
        self.source = source
        self._timeout1 = 60
        self.__timeout2 = 120

    def connect(self):
        print(f"connecting with {self._timeout}s")

conn = Connector("postgresql://localhost")
conn.connect()  # connecting with 60s
conn._timeout1  # 60
conn.__timeout2  # AttributeError: 'Connector' object has no attribute '__timeout2'
conn.Connector.__timeout2  # 120
```

### 프로퍼티
프로퍼티는 아래의 두 경우처럼 객체의 어떤 속성에 대한 접근을 제어하기 위해 사용한다.
- 객체에 값을 저장해야 하는 경우
- 객체의 상태나 다른 속성의 값으로 어떤 계산을 하려고 하는 경우

타 언어에서 `getter`와 `setter`를 만드는 것과 동일하다.

`@property` 데코레이터는 일반적인 `getter`와 역할이 같으며

`@<property>.setter` 데코레이터는 `setter`와 역할이 같다.

In [3]:

class User:
    def __init__(self, username):
        self.username = username
        self._email = None


    def is_valid_email(self, new_email):
        return True if '@' in new_email else False
    
    @property
    def email(self):
        return self._email
    
    @email.setter
    def email(self, new_email):
        if not self.is_valid_email(new_email):
            raise ValueError("유효한 이메일이 아니므로 사용할 수 없음.")
        self._email = new_email

user = User("clean_user")
user.email = "clean_user"

ValueError: 유효한 이메일이 아니므로 사용할 수 없음.

In [ ]:
user.email = "clean_user@test.com"
user.email

'clean_user@test.com'

### 프로퍼티의 장점
프로퍼티를 사용하면 명령-쿼리 분리 원칙(command and query separation)을 따르기도 용이하다.

`@property` 데코레이터는 응답을 위한 query이며

`@<property>.setter` 데코레이터는 무언가를 하기 위한 command이다.

### 파이썬에서의 프로퍼티

일반적인 프로그래밍 언어는 `public`, `private`, `protected` 세 가지 프로퍼티를 가지지만

**파이썬은 모든 프로퍼티와 함수가 `public`하다.** 따라서 호출자가 **모든 객체의 속성을 호출할 수 있다.**

`밑줄(under score)`을 사용하여 다른 언어처럼 `private`를 의미할 수 있지만, 여전히 호출은 가능하다.

## 이터러블 객체

이터러블 객체를 살펴보기에 앞서서 이터러블과 이터레이터를 구분해보자.
- 이터러블 : `__iter__` 매직 메소드를 구현한 객체
- 이터레이터 : `__next__` 매직 메소드를 구현한 객체

파이썬은 기본적으로 반복가능한 리스트, 튜플 등이 있다.

`for e in myobect:` 형태로 객체를 반복할 수 있는지 확인하기 위해서 객체가 `__iter__`나 `__next__`중 하나를 포함하는지와

객체가 시퀀스이고 `__len__`과 `__getitem__`을 모두 가졌는지를 검사한다.

## 컨테이너 객체
`__contains__()` 메소드를 구현한 객체. 일반적으로 Bool값을 반환하도록 구현된다.

해당 키워드는 `in`키워드가 발견될 때 호출된다.

### 사용 예시
x, y를 멤버변수로 가지고 있는 coord가 그리드의 영역에 있는지 검사하고 표시하고 싶을 때, 일반적인 구현은 다음과 같다.

In [4]:
# less Pythonic
def mark_coordinate(grid, coord):
    if 0 <= coord.x < grid.width and 0 <= coord.y < grid.height:
        grid[coord] = "MARKED"

직관적으로 X와 Y가 무엇을 하는지 알아보기 어렵다.

In [5]:
# more pythonic
class Boundaries:
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def __contains__(self, coord):
        x, y = coord
        return 0 <= x < self.width and 0 <= y < self.height
    
class Grid:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.limit = Boundaries(width, height)  # 의도를 직관적으로 설명한다.

    def __contains__(self, coord):
        return coord in self.limits

In [6]:
# Usage
def mark_coordinate(grid, coord):
    if coord in grid:
        grid[coord] = "MARKED"

Pythonic한 코드로 변경한 이후 좀더 직관적으로 변했다.

`in`절을 통해서 `Grid`안에 있는지 체크하는 듯한 느낌을 받기 때문이다.

### 장점
1. 외부에서 사용할 때 해당 코드들은 마치 파이썬이 문제를 해결한 것 처럼 보임
2. 구성이 간단하고 위임을 통해 문제를 해결함(객체들이 모두 최소한의 논리를 사용)

## 객체의 동적 속성
동적으로 어떤 값을 자료구조에 담고 싶으면 그에 좋은 객체가 이미 파이썬에 존재한다. 바로 `Dictionary`이다.

In [7]:
car_info = dict()
car_info['color'] = 'red'
car_info['name'] = 'Ferrari'

print(car_info['color'])
print(car_info['name'])

red
Ferrari


딕셔너리는 불변의 값, 즉 immutable한 자료를 키로 사용하여 자료를 저장하는 해쉬맵 자료구조이다.

> 여기서 해시맵이란, Key와 Value의 쌍을 저장하는 자료구조이다. 또한, 파이썬의 딕셔너리는 매우 효율적이라고 알려져 있어 알고리즘에서도 자주 활용하는 편이다. 특히 동적인 자료구조를 저장하는데 있어 딕셔너리(해시맵)의 경우 다른 자료구조에서 거의 유일하게 `key`라는 값을 이용해서 사용자가 제어할 수 있는 자료구조이니 잘 알아두자.

단, 위의 예시는 사물을 구현한 것이기에 딕셔너리에 저장하기 보단, 다음과 같이 클래스로 구현하면 여러 메소드를 구현할 수 있다.

In [8]:
class Car:
    def __init__(self, name, color):
        self.name = name
        self.color = color

my_car = Car("Ferrari", "Red")
print(f"name : {my_car.name}, color : {my_car.color}")

name : Ferrari, color : Red


클래스로 정의하니 확실히 가독성이 좋아지고, 메소드의 확장또한 충분해 보인다. 그렇다면 인스턴스의 멤버변수는 파이썬이 어떻게 알아내는가?

### `__getattr__`
`my_car.name`을 호출하면 파이썬은 객체의 사전에서 name을 찾아 `__getattribute__`를 호출한다. 객체에 찾고 있는 속성이 없는 경우 속성 이름을 파라미터로 전달하여 `__getattr__('name')`을 호출한다.

이 값을 사용하면 반환 값을 제어할 수도 있고, 심지어는 새로운 속성을 만들어 낼 수 있다.

In [9]:
class DynamicAttributes:
    def __init__(self, attribute):
        self.attribute = attribute

    def __getattr__(self, attr):
        if attr.startswith("fallback_"):
            name = attr.replace("fallback_", "")
            return f"[fallback resolved] {name}"
        raise AttributeError(f"{self.__class__.__name__}에는 {attr}속성이 없음.")


### 호출
attribute 호출

In [10]:
dyn = DynamicAttributes("value")
dyn.attribute  # attribute 호출

'value'

멤버 변수에 없는 값을 호출

> 내부적으로 `dyn.__getattr__('fallback_test')`가 호출되어 처리된다.

In [11]:
dyn.fallback_test

'[fallback resolved] test'

새로운 멤버 변수를 동적으로 생성하기

In [12]:
dyn.__dict__['fallback_new'] = "new_value"
dyn.fallback_new

'new_value'

`__getattr__`은 파이썬 내장 함수 `getattr()`에도 영향을 미친다.

> 파이썬 내장함수 `getattr(instance, attribute_name, default_value)`는 `instance`의 멤버변수에 `attribute_name`이라는 이름을 가진 멤버변수가 있는지를 검사하고, 없는 경우 `default_value`를 반환하는 함수이다.

내장함수 `getattr`은 `dyn.something`을 호출하고, `__getattr__('something')`을 실행시킨다. 단, `something`은 `fallback_`으로 시작하지 않기 때문에 Attribute Error가 발생하고, `getattr`은 `AttributeError`를 감지하여 `default_value`로 설정된 `apple`을 반환한다.

In [13]:
getattr(dyn, "something", "apple")

'apple'

내부에 정의한 `if attr.startswith("fallback_"):`의 조건을 통과하였기 때문에 값을 반환한 `__getattr__`함수는 `AttributeError`를 야기하지 않는다.

In [14]:
getattr(dyn, "fallback_hello" , "apple" )

'[fallback resolved] hello'

### 호출형(callable)객체
클래스 명 뒤에 괄호를 붙이면 생성자 메소드인 `__init__()`이 실행되며, 결과물인 `instance`를 반환한다.

`__call__`함수를 구현하면 `instance`변수 뒤에 괄호가 올 경우 동작을 정의할 수 있다.

In [15]:
from collections import defaultdict

class CallCount:
    def __init__(self):
        self._counts = defaultdict(int)
    def __call__(self, argument):
        self._counts[argument] += 1
        return self._counts[argument]
    
# 호출
cc = CallCount()
print(cc(1))
print(cc(2))
print(cc(1))
cc("something")

1
1
2


1

## 파이썬에서 피해야 할 점
해당 부분은 비교적 쉽게 발견할 수 있다.

### 변경 가능한(mutable)파라미터 기본 값
파라미터에 디폴트값을 설정하는 것은 매번 파라미터를 설정해야 하는 개발자의 피로감을 줄여준다. 다만, 수정이 가능한 변수로 설정하면 문제가 될 수 있다.